https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [ ]:
# Veamos la data que tenemos
!ls ../input

In [ ]:
!ls -U ../input/train/ | tail -n 5

In [ ]:
!ls ../input/test/ | tail -n 5

# Cargando las imagenes

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
from pathlib import Path
%matplotlib inline

In [ ]:
train_path = Path('../input/train/')
cat_imgs, dog_imgs = [], []
for e in train_path.iterdir():
    if 'cat' in e.name: cat_imgs.append(e)
    else              : dog_imgs.append(e)
len(cat_imgs), len(dog_imgs)

In [ ]:
# Definimos una funcion para leer una imagen y hacer el preprocesamiento
from keras.applications.resnet50 import preprocess_input
img_size = 224

def read_img(path):
    x = Image.open(path)
    x = x.resize((img_size, img_size))
    x = np.asarray(x, np.float32)
    return preprocess_input(x)

In [ ]:
# Cargamos una muestra de imagenes para train y validation

# Primero inicializamos los arrays que vamos a usar
x_train = np.ndarray(shape=(2000, img_size, img_size, 3), dtype=np.float32)
y_train = np.zeros(shape=(2000), dtype=np.int8)
x_val = np.ndarray(shape=(200, img_size, img_size, 3), dtype=np.float32)
y_val = np.zeros(shape=(200), dtype=np.int8)

In [ ]:
# Cargamos el train set
for i,e in tqdm_notebook(enumerate(cat_imgs[:1000] + dog_imgs[:1000])):
    x_train[i] = read_img(e)

y_train[1000:] = 1 # cat -> 0 | dog -> 1

In [ ]:
# Cargamos el validation set
for i,e in tqdm_notebook(enumerate(cat_imgs[1000:1100] + dog_imgs[1000:1100])):
    x_val[i] = read_img(e)

y_val[100:] = 1 # cat -> 0 | dog -> 1

# Cargamos una red entrenada
Vamos a cargar la red **ResNet50** ya entrenada, pero sin incluir las capas densas, ya que vamos a adaptar la red a nuestro caso específico.

In [ ]:
from keras.applications.resnet50 import ResNet50
base_model = ResNet50(include_top=False, input_shape=(img_size,img_size,3), pooling='avg')
base_model.summary()

In [ ]:
base_model.input, base_model.output

In [ ]:
# Dado que solo queremos entrenar las capas densas del modelo que agregaremos
# en el siguiente paso, vamos a setear "trainable = False" para que los pesos
# de la red entrenada no cambien.
base_model.trainable = False

# Creamos el modelo clasificador

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

top_model = Sequential([
    Dense(128, activation='relu', input_shape=(2048,)),
    Dense(1, activation='sigmoid')
])

top_model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
top_model.summary()

# Juntamos los 2 modelos

In [ ]:
final_model = Sequential([base_model, top_model])

final_model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
final_model.summary()

# Entrenando en nuestro dataset

In [ ]:
# log = final_model.fit(x_train, y_train, batch_size=64, validation_data=[x_val, y_val])

La red de base que estamos usando (ResNet50) tiene bastantes capas y hacer todas estas operaciones toma un tiempo considerable, en CPU hacer este entrenamiento puede resultar impractico.
# Precompute
Si tenemos en cuenta que vamos a entrenar nuestro dataset un cierto número de épocas, en cada época se van a repetir exactamente las mismas operaciones en la misma data. Para no redundar, es bastante útil hacer un precompute de la data:
1. Pasar todas nuestras imagenes por la red base (ResNet50).
2. Guardamos los features extraidos.
3. Entrenamos las capas densas con los features extraidos.

In [ ]:
precomputed_train = base_model.predict(x_train, batch_size=128, verbose=1)
precomputed_train.shape

In [ ]:
precomputed_val = base_model.predict(x_val, batch_size=128, verbose=1)
precomputed_val.shape

# Entrenar a partir de los features extraidos

In [ ]:
# Ahora podemos usar un batch_size mas grande, ya que los features son mas pequeños
# que las imagenes.
log = top_model.fit(precomputed_train, y_train, epochs=5, batch_size=256, validation_data=[precomputed_val, y_val])

In [ ]:
def show_results(log):
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(log.history['val_loss'], label='validation')
    ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
    ax2.plot(log.history['acc'], label='train')
    ax2.plot(log.history['val_acc'], label='validation')
    ax2.set_xlabel('epoch'); ax2.set_ylabel('accuracy')
    for ax in axes: ax.legend()

In [ ]:
show_results(log)

# Ejercicio: usando el modelo completo en el test set

In [ ]:
test_path = Path('../input/test/')
test_files = list(test_path.iterdir())

In [ ]:
def get_class(path):
    # Cargar la imagen del path
    img = Image.open(path)
    
    # Cambiar el tamaño de la imagen
    img_resized = img.resize((224, 224))
    
    # Cambiar a formato numpy y preprocesar
    x = np.asarray(img_resized, np.float32)[None]
    x = preprocess_input(x)
    
    # Obtener predicciones
    y = final_model.predict(x)
    
    # Decodear predicciones
    pred = 'cat' if y < 0.5 else 'dog'
    
    # Mostrar la imagen
    plt.imshow(img)
    plt.axis('off')
    plt.title(pred, size=14)
    
    return

In [ ]:
sample = np.random.choice(test_files)
get_class(sample)